### Plot metamap for the resulted embedding using tsne

In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
from sklearn.manifold import TSNE
from dataset_utils import load_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Read all calculated embedding from pickle file into memory

In [7]:
dataset_name = 'COIL20'
embedding_dir = './output/{}'.format(dataset_name)
X, y, labels = load_dataset(dataset_name)

Loading dataset: COIL20


In [8]:
# load calculated embedding for different value of perplexity
import joblib
import os

embeddings = []
perps = []
all_losses = []

for file in os.listdir(embedding_dir):
    if file.endswith('.z'):
        in_name = os.path.join(embedding_dir, file)
        tsne_obj = joblib.load(in_name)
        embeddings.append(tsne_obj.embedding_.ravel())
        all_losses.append(tsne_obj.kl_divergence_)
        perps.append(tsne_obj.get_params()['perplexity'])
        
embeddings = np.array(embeddings)

In [ ]:
# build metamap with different perplexities
from matplotlib import pyplot as plt
%matplotlib inline

meta_perps = [2, 5, 10, 20, 30, 35, 40, 45, 50, 75, 100, 150]
meta_maps = []

for meta_perp in meta_perps:
    a_meta_map = TSNE(perplexity=meta_perp, random_state=0).fit_transform(embeddings)
    meta_maps.append(a_meta_map)

In [ ]:
# plot metamap with 
mfig, maxes = plt.subplots(3, 4, figsize=(20, 15))

for i, meta_perp in enumerate(meta_perps):
    r = i//4
    c = i%4
    map2d = meta_maps[i]
    maxes[r][c].scatter(map2d[:,0], map2d[:,1], c=np.log(perps))
    maxes[r][c].set_title('metamap perp={}'.format(meta_perp))
mfig.suptitle('Metamap for {} \n (Colored by log of perplexity)'.format(dataset_name))

plt.savefig('./plots/[{}]metamap'.format(dataset_name))
plt.show()

In [ ]:
meta_tsne = TSNE(perplexity=40, random_state=0)
meta_map = meta_tsne.fit_transform(embeddings)
print(meta_tsne)

In [ ]:
from bqplot import LinearScale, ColorScale, OrdinalColorScale, Axis, ColorAxis, Scatter, Figure
from bqplot.colorschemes import CATEGORY10, CATEGORY20
from ipywidgets import VBox, HBox

# meta scatter
sc_color = ColorScale()
ax_color = ColorAxis(scale=sc_color, label='Log of Perplexity', orientation='vertical', side='bottom')
scatter = Scatter(x=meta_map[:, 0], y=meta_map[:, 1], color=np.log(perps), stroke='black',
                  scales={'x': LinearScale(), 'y': LinearScale(), 'color': sc_color})

# child scatter
sc_color2 = OrdinalColorScale(colors=CATEGORY20)
ax_color2 = ColorAxis(scale=sc_color2, label='Class', orientation='horizontal', side='right')
child_scatter = Scatter(x=[], y=[], color=y, scales={'x': LinearScale(), 'y': LinearScale(), 'color': sc_color2}, default_size=12)

scat_fig = Figure(axes=[ax_color], marks=[scatter], title='Metamap for {}'.format(dataset_name))
child_fig = Figure(axes=[ax_color2],marks=[child_scatter], title='Detail view')

In [ ]:
def plot_detail(name, value):
    idx = value['data']['index']
    perp = perps[idx]
    X2d = all_embeddings[idx].reshape(-1, 2)
    # with child_scatter.hold_sync():
    child_scatter.x = X2d[:, 0]
    child_scatter.y = X2d[:, 1]
    child_fig.title = 'Detail scatter for perp = {}'.format(perp)
    
scatter.on_element_click(plot_detail)

In [ ]:
HBox([scat_fig, child_fig])